In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv


## Loading datasets

In [3]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv', index_col='id')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv', index_col='id')
display(train_df)
display(test_df)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
id,,,,,,,,,,,,
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
58640,34,120000,MORTGAGE,5.0,EDUCATION,D,25000,15.95,0.21,Y,10,0
58641,28,28800,RENT,0.0,MEDICAL,C,10000,12.73,0.35,N,8,1
58642,23,44000,RENT,7.0,EDUCATION,D,6800,16.00,0.15,N,2,1


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
id,,,,,,,,,,,
58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4
...,...,...,...,...,...,...,...,...,...,...,...
97738,22,31200,MORTGAGE,2.0,DEBTCONSOLIDATION,B,3000,10.37,0.10,N,4
97739,22,48000,MORTGAGE,6.0,EDUCATION,A,7000,6.03,0.15,N,3
97740,51,60000,MORTGAGE,0.0,PERSONAL,A,15000,7.51,0.25,N,25


## Looking for missing values

In [4]:
missing_train = train_df.isnull().sum()
missing_test = test_df.isnull().sum()

missing_value_cols1 = missing_train[missing_train>0]
missing_value_cols2 = missing_test[missing_test>0]
missing_value_df = pd.DataFrame(missing_value_cols1)
missing_value_df['test_df'] = missing_value_cols2
missing_value_df

,0,test_df


## Looking for duplicate values

In [4]:
train_df.duplicated().sum()

0

## EDA

In [5]:
train_df.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,58645.000000,5.864500e+04,58645.000000,58645.000000,58645.000000,58645.000000,58645.000000,58645.000000
mean,27.550857,6.404617e+04,4.701015,9217.556518,10.677874,0.159238,5.813556,0.142382
std,6.033216,3.793111e+04,3.959784,5563.807384,3.034697,0.091692,4.029196,0.349445
min,20.000000,4.200000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,0.000000
25%,23.000000,4.200000e+04,2.000000,5000.000000,7.880000,0.090000,3.000000,0.000000
50%,26.000000,5.800000e+04,4.000000,8000.000000,10.750000,0.140000,4.000000,0.000000
75%,30.000000,7.560000e+04,7.000000,12000.000000,12.990000,0.210000,8.000000,0.000000
max,123.000000,1.900000e+06,123.000000,35000.000000,23.220000,0.830000,30.000000,1.000000


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58645 entries, 0 to 58644
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  58645 non-null  int64  
 1   person_income               58645 non-null  int64  
 2   person_home_ownership       58645 non-null  object 
 3   person_emp_length           58645 non-null  float64
 4   loan_intent                 58645 non-null  object 
 5   loan_grade                  58645 non-null  object 
 6   loan_amnt                   58645 non-null  int64  
 7   loan_int_rate               58645 non-null  float64
 8   loan_percent_income         58645 non-null  float64
 9   cb_person_default_on_file   58645 non-null  object 
 10  cb_person_cred_hist_length  58645 non-null  int64  
 11  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 5.8+ MB


In [7]:
train_df.columns

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file',
       'cb_person_cred_hist_length', 'loan_status'],
      dtype='object')

In [8]:
for col in train_df[['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file',
       'cb_person_cred_hist_length']]:
    print("column : ",col,"\nunique values : ",train_df[col].unique(),"\n")

column :  person_age 
unique values :  [ 37  22  29  30  27  25  21  35  31  26  28  32  23  24  33  44  38  43
  46  48  36  41  34  40  47  42  51  50  39  56  60  49  58  54  45  59
  65  53  55  57  52  62  64  84  70  66  73  69  61  20  80  76 123] 

column :  person_income 
unique values :  [ 35000  56000  28800 ...  44955  90730 124600] 

column :  person_home_ownership 
unique values :  ['RENT' 'OWN' 'MORTGAGE' 'OTHER'] 

column :  person_emp_length 
unique values :  [  0.   6.   8.  14.   2.   9.  11.   5.   1.  13.   3.  10.  12.   4.
   7.  21.  17.  16.  18.  19.  15.  20.  23.  24.  26.  22.  25.  41.
  27.  28.  35.  29.  30.  31. 123.  39.] 

column :  loan_intent 
unique values :  ['EDUCATION' 'MEDICAL' 'PERSONAL' 'VENTURE' 'DEBTCONSOLIDATION'
 'HOMEIMPROVEMENT'] 

column :  loan_grade 
unique values :  ['B' 'C' 'A' 'D' 'E' 'F' 'G'] 

column :  loan_amnt 
unique values :  [ 6000  4000 12000  9000  2500  5000 15000 12800 10000 16000  7300 17000
  5950  4800  3200  7000 

## Feature selection and splitting

In [7]:
from sklearn.model_selection import train_test_split
X = train_df.drop(['loan_status'], axis=1)
y = train_df['loan_status']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, test_size=0.2, random_state =365)

## Preprocessing

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

rs = RobustScaler()
ohe = OneHotEncoder(sparse_output=False)

ct = make_column_transformer(
    (rs, make_column_selector(dtype_exclude = object)),
    (ohe, make_column_selector(dtype_include = object)),
    remainder='passthrough')
ct.fit_transform(X_train)

array([[ 1.57142857, -0.45454545, -0.8       , ...,  0.        ,
         1.        ,  0.        ],
       [-0.28571429, -0.24242424,  0.4       , ...,  0.        ,
         1.        ,  0.        ],
       [ 4.57142857, -0.63636364, -0.8       , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.42857143,  1.87878788, -0.8       , ...,  0.        ,
         1.        ,  0.        ],
       [-0.57142857, -0.18181818,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.28571429,  1.27272727, -0.4       , ...,  0.        ,
         1.        ,  0.        ]])

## Creating pipeline and comparing different model

In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint
import optuna
from sklearn.model_selection import cross_val_score

In [10]:
LR = LogisticRegression()

pipe_1 = make_pipeline(ct, LR)

cv_score = cross_val_score(pipe_1, X_train, y_train, cv=5, scoring='roc_auc').mean()
print("Best cross-validation score for LR: {:.4f}".format(cv_score))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Best cross-validation score for LR: 0.9007


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
%%time

def objective(trial):
    param = {
    'max_depth': trial.suggest_int('max_depth', 2, 10),
    'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
    'n_estimators': trial.suggest_int('n_estimators', 50, 300),
    'subsample': trial.suggest_float('subsample', 0.5, 1.0),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    'gamma': trial.suggest_float('gamma', 0, 5),
    'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
    'reg_lambda': trial.suggest_float('reg_lambda', 0, 5)}
    
    pipeline = make_pipeline(ct,XGBClassifier(**param, random_state=365))
    
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='roc_auc')
    return scores.mean()
    
    
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)


[I 2024-10-12 18:12:54,951] A new study created in memory with name: no-name-6c7eebbf-0ac7-45c2-ab4d-dbbf4bd8c733
[I 2024-10-12 18:12:59,840] Trial 0 finished with value: 0.9272605705613651 and parameters: {'max_depth': 5, 'learning_rate': 0.008383418339293899, 'n_estimators': 228, 'subsample': 0.5489953146928617, 'colsample_bytree': 0.8806384459394807, 'gamma': 3.124335667228672, 'reg_alpha': 4.532975530018183, 'reg_lambda': 3.8148687814248023}. Best is trial 0 with value: 0.9272605705613651.
[I 2024-10-12 18:13:03,242] Trial 1 finished with value: 0.9207088792069975 and parameters: {'max_depth': 5, 'learning_rate': 0.0041594534035491815, 'n_estimators': 162, 'subsample': 0.8103250963533601, 'colsample_bytree': 0.9733563170263824, 'gamma': 1.3106898794788697, 'reg_alpha': 3.1156556460947176, 'reg_lambda': 0.7522344232294276}. Best is trial 0 with value: 0.9272605705613651.
[I 2024-10-12 18:13:05,348] Trial 2 finished with value: 0.9472988171677216 and parameters: {'max_depth': 5, 'lea

CPU times: user 30min 25s, sys: 11 s, total: 30min 36s
Wall time: 8min 28s


In [19]:
best_params = study.best_params

opt_XGBC = XGBClassifier(**best_params)

pipe_2 = make_pipeline(ct, opt_XGBC)

pipe_2.fit(X_train, y_train)

XGB_pred_proba = pipe_2.predict_proba(X_valid)[:, 1]

XGB_roc_auc = roc_auc_score(y_valid, XGB_pred_proba)
print("ROC AUC Score for opt_XGBC: {:.4f}".format(XGB_roc_auc))

ROC AUC Score for opt_XGBC: 0.9569


In [20]:
%%time


def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),}

    pipeline = make_pipeline(ct, lgb.LGBMClassifier(**param))

    pipeline.fit(X_train, y_train)
    preds = pipeline.predict_proba(X_valid)[:, 1]
    roc_auc = roc_auc_score(y_valid, preds)
    return roc_auc

    return accuracy

study_2 = optuna.create_study(direction='maximize')
study_2.optimize(objective, n_trials=50)

[I 2024-10-12 18:54:18,030] A new study created in memory with name: no-name-3380b7a8-9926-4a17-aa28-812c8d7d6047
[I 2024-10-12 18:54:19,429] Trial 0 finished with value: 0.9551569690919385 and parameters: {'lambda_l1': 5.763956688463779e-07, 'lambda_l2': 2.2141937139452675e-07, 'num_leaves': 53, 'feature_fraction': 0.673532170238416, 'bagging_fraction': 0.4533034174125431, 'bagging_freq': 3, 'min_child_samples': 64}. Best is trial 0 with value: 0.9551569690919385.
[I 2024-10-12 18:54:20,083] Trial 1 finished with value: 0.9302896146269941 and parameters: {'lambda_l1': 1.0682436459738876e-06, 'lambda_l2': 2.0143443454547068e-07, 'num_leaves': 3, 'feature_fraction': 0.6574089838829806, 'bagging_fraction': 0.4308476197389403, 'bagging_freq': 3, 'min_child_samples': 71}. Best is trial 0 with value: 0.9551569690919385.
[I 2024-10-12 18:54:21,285] Trial 2 finished with value: 0.9559272150598892 and parameters: {'lambda_l1': 0.017337638221514712, 'lambda_l2': 3.5190513984332205e-06, 'num_lea

CPU times: user 1min 24s, sys: 71.3 ms, total: 1min 24s
Wall time: 1min 24s


In [21]:
print("Best parameters:", study_2.best_params)

Best parameters: {'lambda_l1': 1.0751751193708883e-07, 'lambda_l2': 0.004543378261357417, 'num_leaves': 54, 'feature_fraction': 0.5806171002398998, 'bagging_fraction': 0.9931429238356542, 'bagging_freq': 2, 'min_child_samples': 58}


In [22]:
best_params = study.best_params

opt_LGBC = lgb.LGBMClassifier(**best_params)

pipe_3 = make_pipeline(ct, opt_LGBC)

pipe_3.fit(X_train, y_train)

LGB_pred_proba = pipe_3.predict_proba(X_valid)[:, 1]

LGB_roc_auc = roc_auc_score(y_valid, LGB_pred_proba)
print("ROC AUC Score for opt_XGBC: {:.4f}".format(LGB_roc_auc))

[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 6680, number of negative: 40236
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011036 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 896
[LightGBM] [Info] Number of data points in the train set: 46916, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142382 -> initscore=-1.795644
[LightGBM] [Info] Start training from score -1.795644
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leave

In [23]:
%%time
def objective(trial):
    param = {
        'objective': 'Logloss',
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.01, 1.0),
        'depth': trial.suggest_int('depth', 1, 12),
        'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'used_ram_limit': '3gb',
    }

    if param['bootstrap_type'] == 'Bayesian':
        param['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
    elif param['bootstrap_type'] == 'Bernoulli':
        param['subsample'] = trial.suggest_float('subsample', 0.1, 1.0)

    pipeline = make_pipeline(ct, CatBoostClassifier(**param, verbose=0)) 

    pipeline.fit(X_train, y_train)
    preds = pipeline.predict_proba(X_valid)[:, 1]
    roc_auc = roc_auc_score(y_valid, preds)
    return roc_auc

# Create a study and optimize the objective function
study_3 = optuna.create_study(direction='maximize')
study_3.optimize(objective, n_trials=20)


[I 2024-10-12 19:12:21,377] A new study created in memory with name: no-name-e0da8a83-01ac-4a24-b190-07c2e7e66de4
[I 2024-10-12 19:13:24,872] Trial 0 finished with value: 0.9598131503173196 and parameters: {'colsample_bylevel': 0.5965931617866997, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.5421712265707523}. Best is trial 0 with value: 0.9598131503173196.
[I 2024-10-12 19:14:15,974] Trial 1 finished with value: 0.942517767923741 and parameters: {'colsample_bylevel': 0.5639049379729502, 'depth': 8, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 7.562188393007073}. Best is trial 0 with value: 0.9598131503173196.
[I 2024-10-12 19:14:22,909] Trial 2 finished with value: 0.9344293220894924 and parameters: {'colsample_bylevel': 0.54703355072631, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 9.261081109108435}. Best is trial 0 with value: 0.9598131503173196.
[I 20

CPU times: user 1h 51min 37s, sys: 27min 55s, total: 2h 19min 32s
Wall time: 47min 32s


In [24]:
print("Best parameters:", study_3.best_params)

Best parameters: {'colsample_bylevel': 0.9989032392859094, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}


In [28]:
best_params = study.best_params

opt_catb = CatBoostClassifier(colsample_bylevel= 0.9989032392859094, depth=6, 
                              boosting_type = 'Ordered', bootstrap_type ='MVS',verbose=0)

pipe_4 = make_pipeline(ct, opt_catb)

pipe_4.fit(X_train, y_train)

catb_pred_proba = pipe_4.predict_proba(X_valid)[:, 1]

catb_roc_auc = roc_auc_score(y_valid, catb_pred_proba)
print("ROC AUC Score for opt_catb: {:.4f}".format(catb_roc_auc))

ROC AUC Score for opt_catb: 0.9621


## Training on best model

In [30]:
pipe_4.fit(X,y)

test_pred_proba = pipe_4.predict_proba(test_df)[:, 1]

In [33]:
submission = pd.read_csv('/kaggle/input/playground-series-s4e10/sample_submission.csv')
submission['loan_status'] = test_pred_proba
submission.head(5)

,id,loan_status
0,58645,0.998939
1,58646,0.025616
2,58647,0.678886
3,58648,0.008466
4,58649,0.032582


In [35]:
submission.to_csv("submission.csv", index=False)